In [ ]:
# !pip install -U tfx
# !pip install apache-beam==2.39.0

In [ ]:
import apache_beam as beam
from google.protobuf import text_format
import math
import matplotlib.pyplot as plt
import os
import pandas as pd
import pathlib
import pprint
import tempfile
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam
from tensorflow.python.lib.io import file_io
from tfx_bsl.public import tfxio
from tfx_bsl.coders.example_coder import RecordBatchToExamples

# Display versions of TF and TFX related packages
print('Beam: {}'.format(beam.__version__))
print('TF: {}'.format(tf.__version__))
print('Transform: {}'.format(tft.__version__))

Beam: 2.39.0
TF: 2.8.2
Transform: 1.8.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Location of the data and model directory
DATA_DIR = '/content/drive/My Drive/Stroke Prediction ML System'
DATA_ROOT = f'{DATA_DIR}/data'

# Set the paths to the reduced dataset
DATA_DIR_SELECT = f'{DATA_DIR}/data/select'
TRAINING_ROOT = f'{DATA_DIR}/training'
TESTING_ROOT = f'{DATA_DIR}/testing'
SERVING_ROOT = f'{DATA_DIR}/serving'

TRAINING_DATA = f'{DATA_DIR}/training/stoke_prediction_training_dataset.csv'
TESTING_DATA = f'{DATA_DIR}/training/stoke_prediction_testing_dataset.csv'
SERVING_DATA = f'{DATA_DIR}/serving/stoke_prediction_serving_dataset.csv'

# We will create two pipelines. One for schema generation and one for training.
SCHEMA_PIPELINE_NAME = 'stroke-mlops-schema'
PIPELINE_NAME = 'stroke-mlops'

# Output directory to store artifacts generated from the pipeline.
SCHEMA_PIPELINE_ROOT = os.path.join(DATA_DIR, 'pipeline', SCHEMA_PIPELINE_NAME)
PIPELINE_ROOT = os.path.join(DATA_DIR, 'pipeline', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
SCHEMA_METADATA_PATH = os.path.join(DATA_DIR, 'metadata', SCHEMA_PIPELINE_NAME, 'metadata.db')
METADATA_PATH = os.path.join(DATA_DIR, 'metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join(DATA_DIR, 'serving')

# Path to curated schema file
SCHEMA_FOLDER = os.path.join(DATA_DIR, 'schema/schema_output')

# Names of transformed data files
TRANSFORMED_TRAIN_DATA = 'train_transformed'
TRANSFORMED_TEST_DATA = 'test_transformed'
OUTPUT_DIR = f'{DATA_ROOT}/transformed'

# Set random seed
RANDOM_SEED = 0

In [ ]:
SCALE_MINMAX_FEATURE_KEYS = ['age']

SCALE_Z_FEATURE_KEYS = ['avg_glucose_level', 'bmi']

VOCAB_FEATURE_KEYS = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

FEATURE_KEYS = ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status', 'stroke']

LABEL_KEY = 'stroke'

In [ ]:
def load_schema(input_path):
  schema = schema_pb2.Schema()
  schema_text = file_io.read_file_to_string(input_path)
  text_format.Parse(schema_text, schema)
  return schema

curated_schema = load_schema(f'{SCHEMA_FOLDER}/schema.pbtxt')

In [ ]:
def preprocessing_fn(inputs):
  """
  Preprocess input columns into transformed columns
  """

  features_dict = {}

  for feature in SCALE_MINMAX_FEATURE_KEYS:
      data_col = inputs[feature] 
      # Transform using scaling of min_max function
      features_dict[feature] = tft.scale_by_min_max(data_col)

  for feature in SCALE_Z_FEATURE_KEYS:
      data_col = inputs[feature] 
      # Transforming using scaling to z score
      features_dict[feature] = tft.scale_to_z_score(data_col)

  for feature in VOCAB_FEATURE_KEYS:
      data_col = inputs[feature] 
      # Transforming using vocabulary available in column
      features_dict[feature] = tft.compute_and_apply_vocabulary(data_col)

  # No change in the label
  features_dict[LABEL_KEY] = inputs[LABEL_KEY]

  return features_dict

In [ ]:
def transform_data(train_data_file, test_data_file, working_dir):
  """
  Transform the data and write out as a TFRecord of Example protos.

  Read in the data using the CSV reader, and transform it using a
  preprocessing pipeline that scales numeric data and converts categorical data
  from strings to int64 values indices, by creating a vocabulary for each
  category.

  Args:
    train_data_file: File containing training data
    test_data_file: File containing test data
    working_dir: Directory to write transformed data and metadata to
  """

  with beam.Pipeline() as pipeline:
    with tft_beam.Context(temp_dir=DATA_ROOT):
      # Create a TFXIO to read the census data with the schema. To do this we
      # need to list all columns in order since the schema doesn't specify the
      # order of columns in the csv
      train_csv_tfxio = tfxio.CsvTFXIO(
          file_pattern=train_data_file,
          skip_header_lines=1,
          telemetry_descriptors=[],
          column_names=FEATURE_KEYS,
          schema=curated_schema)

      # Read in raw data and convert using CSV TFXIO
      raw_data = (
          pipeline |
          'ReadTrainCsv' >> train_csv_tfxio.BeamSource())

      # Combining data and schema into a dataset tuple.  Note that we already used
      # the schema to read the CSV data, but we also need it to interpret
      # raw_data
      cfg = train_csv_tfxio.TensorAdapterConfig()
      raw_dataset = (raw_data, cfg)

      # The TFXIO output format is chosen for improved performance
      transformed_dataset, transform_fn = (
          raw_dataset | tft_beam.AnalyzeAndTransformDataset(
              preprocessing_fn, output_record_batches=True))

      # Transformed metadata is not necessary for encoding.
      transformed_data, _ = transformed_dataset

      # Extract transformed RecordBatches, encode and write them to the given
      # directory
      _ = (
          transformed_data
          | 'EncodeTrainData' >>
          beam.FlatMapTuple(lambda batch, _: RecordBatchToExamples(batch))
          | 'WriteTrainData' >> beam.io.WriteToTFRecord(
              os.path.join(working_dir, TRANSFORMED_TRAIN_DATA)))

      # Now applying transform function to test data.  In this case we also 
      # remove the header line that is present in the test data file
      test_csv_tfxio = tfxio.CsvTFXIO(
          file_pattern=test_data_file,
          skip_header_lines=1,
          telemetry_descriptors=[],
          column_names=FEATURE_KEYS,
          schema=curated_schema)
      
      raw_test_data = (
          pipeline
          | 'ReadTestCsv' >> test_csv_tfxio.BeamSource())

      raw_test_dataset = (raw_test_data, test_csv_tfxio.TensorAdapterConfig())

      # The TFXIO output format is chosen for improved performance
      transformed_test_dataset = (
          (raw_test_dataset, transform_fn)
          | tft_beam.TransformDataset(output_record_batches=True))

      # Transformed metadata is not necessary for encoding
      transformed_test_data, _ = transformed_test_dataset

      # Extract transformed RecordBatches, encode and write them to the given
      # directory
      _ = (
          transformed_test_data
          | 'EncodeTestData' >>
          beam.FlatMapTuple(lambda batch, _: RecordBatchToExamples(batch))
          | 'WriteTestData' >> beam.io.WriteToTFRecord(
              os.path.join(working_dir, TRANSFORMED_TEST_DATA)))

      # Will write a SavedModel and metadata to working_dir, which can then
      # be read by the tft.TFTransformOutput class
      _ = (
          transform_fn
          | 'WriteTransformFn' >> tft_beam.WriteTransformFn(working_dir))

In [ ]:
transform_data(TRAINING_DATA, TESTING_DATA, OUTPUT_DIR)

INFO:absl:Feature Residence_type has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature avg_glucose_level has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature bmi has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature ever_married has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature gender has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature heart_disease has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature hypertension has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature smoking_status has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature stroke has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature work_type has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature Residence_type has a shape dim {
  size: 1
}
. S

In [ ]:
tf_transform_output = tft.TFTransformOutput(OUTPUT_DIR)
tf_transform_output.transformed_feature_spec()

{'Residence_type': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None),
 'age': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'avg_glucose_level': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'bmi': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'ever_married': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None),
 'gender': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None),
 'smoking_status': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None),
 'stroke': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None),
 'work_type': FixedLenFeature(shape=[1], dtype=tf.int64, default_value=None)}

In [ ]:
# Set the constants filename and module filename
_stroke_constants_file = 'stroke_constants.py'
_stroke_module_file = 'stroke_module.py'

In [ ]:
%%writefile {_stroke_constants_file}

SCALE_MINMAX_FEATURE_KEYS = ['age']

SCALE_Z_FEATURE_KEYS = ['avg_glucose_level', 'bmi']

VOCAB_FEATURE_KEYS = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

LABEL_KEY = 'stroke'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Writing stroke_constants.py


In [ ]:
%%writefile {_stroke_module_file}

import os
import pickle
from typing import List, Text, Tuple
from absl import logging
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow_metadata.proto.v0 import schema_pb2
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils
import tensorflow_datasets as tfds
from tfx.components.trainer.fn_args_utils import DataAccessor, FnArgs
from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tfx.utils import io_utils
from tfx.dsl.io import fileio
from tfx_bsl.tfxio import dataset_options
import stroke_constants

logging.set_verbosity(logging.INFO) 

_SCALE_MINMAX_FEATURE_KEYS = stroke_constants.SCALE_MINMAX_FEATURE_KEYS
_SCALE_Z_FEATURE_KEYS = stroke_constants.SCALE_Z_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = stroke_constants.VOCAB_FEATURE_KEYS
_FEATURE_KEYS = _SCALE_MINMAX_FEATURE_KEYS + _SCALE_Z_FEATURE_KEYS + _VOCAB_FEATURE_KEYS
_LABEL_KEY = stroke_constants.LABEL_KEY
_transformed_name = stroke_constants.transformed_name

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

def preprocessing_fn(inputs):

    features_dict = {}

    for feature in _SCALE_MINMAX_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transform using scaling of min_max function
        features_dict[_transformed_name(feature)] = tft.scale_by_min_max(data_col)

    for feature in _SCALE_Z_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transforming using scaling to z score
        features_dict[_transformed_name(feature)] = tft.scale_to_z_score(data_col)

    for feature in _VOCAB_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transforming using vocabulary available in column
        features_dict[_transformed_name(feature)] = tft.compute_and_apply_vocabulary(data_col)

    # No change in the label
    features_dict[_LABEL_KEY] = inputs[_LABEL_KEY]

    return features_dict

def _apply_preprocessing(raw_features, tft_layer):

  transformed_features = tft_layer(raw_features)
  
  if _LABEL_KEY in raw_features:
    transformed_label = transformed_features.pop(_LABEL_KEY)
    return transformed_features, transformed_label
  
  else:
    return transformed_features, None

def _input_fn(file_pattern: List[Text], data_accessor: tfx.components.DataAccessor, tf_transform_output: tft.TFTransformOutput, batch_size: int = 200) -> tf.data.Dataset:
  """
  Generates features and label for tuning/training

  Args:
    file_pattern: List of paths or patterns of input tfrecord files
    data_accessor: DataAccessor for converting input to RecordBatch
    tf_transform_output: A TFTransformOutput
    batch_size: representing the number of consecutive elements of returned dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a dictionary of Tensors, and indices is a single Tensor of label indices.
  """

  dataset = data_accessor.tf_dataset_factory(file_pattern, tfxio.TensorFlowDatasetOptions(batch_size=batch_size, label_key=_LABEL_KEY), schema=tf_transform_output.raw_metadata.schema)

  transform_layer = tf_transform_output.transform_features_layer()
  def apply_transform(raw_features):
    return _apply_preprocessing(raw_features, transform_layer)

  return dataset.map(apply_transform)


Overwriting stroke_module.py


In [ ]:
def create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                    schema_path: str, module_file: str, serving_model_dir: str, 
                    metadata_path: str) -> tfx.dsl.Pipeline:
  """
  Creates a pipeline for data validation, feature engineering, modeling, and model analysis
  Args:
    pipeline_name: name of schema pipeline
    pipeline_root: path of schema pipeline directory
    data_root: path of data file directory
    schema_path: path of curated schema
    metadata_path: path of metadata

  Returns:
    Pipeline with example_gen, statistics_gen, and schema_gen
  """

  # Brings data into the pipeline
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Computes statistics over data for visualization and schema generation
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])
  
  # Import the schema.
  schema_importer = tfx.dsl.Importer(
      source_uri=schema_path,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer')

  # Performs anomaly detection based on statistics and data schema.
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_importer.outputs['result'])
  
  transform = tfx.components.Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_importer.outputs['result'],
      materialize=False,
      module_file=module_file)
  
    # tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
    # # schema = tfx.utils.parse_pbtxt_file(fn_args.schema_path, schema_pb2.Schema())

    # train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, tf_transform_output, batch_size=_TRAIN_BATCH_SIZE)
    # eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, tf_transform_output, batch_size=_EVAL_BATCH_SIZE)

    # train_ds = pd.DataFrame(list(train_dataset.as_numpy_iterator()))
    # eval_ds = pd.DataFrame(list(train_dataset.as_numpy_iterator()))

  # # Uses user-provided Python function that trains a model
  # trainer = tfx.components.Trainer(
  #     module_file=module_file,
  #     examples=example_gen.outputs['examples'],
  #     schema=schema_importer.outputs['result'],
  #     # Pass transform_graph to the trainer
  #     transform_graph=transform.outputs['transform_graph'])

  # # Pushes the model to a filesystem destination.
  # pusher = tfx.components.Pusher(
  #     model=trainer.outputs['model'],
  #     push_destination=tfx.proto.PushDestination(
  #         filesystem=tfx.proto.PushDestination.Filesystem(base_directory=serving_model_dir)))
  
  components = [example_gen, 
                statistics_gen, 
                schema_importer, 
                example_validator, 
                transform]

  return tfx.dsl.Pipeline(pipeline_name=pipeline_name,
                          pipeline_root=pipeline_root,
                          metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(metadata_path),
                          components=components)

In [ ]:
tfx.orchestration.LocalDagRunner().run(create_pipeline(pipeline_name=PIPELINE_NAME,
                                                       pipeline_root=PIPELINE_ROOT,
                                                       data_root=TRAINING_ROOT,
                                                       schema_path=SCHEMA_FOLDER,
                                                       module_file=_stroke_module_file,
                                                       serving_model_dir=SERVING_MODEL_DIR,
                                                       metadata_path=METADATA_PATH))

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Generating ephemeral wheel package for '/content/stroke_module.py' (including modules: ['stroke_module', 'stroke_constants']).
INFO:absl:User module package has hash fingerprint version 9f1220e851e705e3d616e2d028c08d83765f89531cc17b67c3cfb63531a1fa15.
INFO:absl:Executing: ['/usr/bin/python3', '/tmp/tmp9rq3yc_z/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp6ennvtlg', '--dist-dir', '/tmp/tmprvo3esnv']
INFO:absl:Successfully built user code wheel distribution at '/content/drive/My Drive/Stroke Prediction ML System/pipeline/stroke-mlops/_wheels/tfx_user_code_Transform-0.0+9f1220e851e705e3d616e2d028c08d83765f89531cc17b67c3cfb63531a1fa15-py3-none-any.whl'; target user module is 'stroke_module'.
INFO:absl:Full user module path is 'stroke_module@/content/drive/My Drive/Stroke Prediction ML System/pipeline/stroke-mlops/_wheels/tfx_user_co

INFO:absl:Processing input csv data /content/drive/My Drive/Stroke Prediction ML System/training/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Value type <class 'NoneType'> of key version in exec_properties is not supported, going to drop it
INFO:absl:Value type <class 'list'> of key _beam_pipeline_args in exec_properties is not supported, going to drop it
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 7 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'examples': [Artifact(artifact: uri: "/content/drive/My Drive/Stroke Prediction ML System/pipeline/stroke-mlops/CsvExampleGen/examples/7"
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:283991,xor_checksum:1656620775,sum_checksum:1656620775"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "stroke-mlops:2022-06-30T20:27:53.743512:CsvExampleGen:

In [ ]:
def _test_schema_pipeline(pipeline_name: str,
                          pipeline_root: str,
                          data_root: str,
                          schema_path: str,
                          metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a pipeline for schema generation."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # NEW: Computes statistics over data for visualization and schema generation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # NEW: Generates schema based on the generated statistics.
  schema_importer = tfx.components.ImportSchemaGen(
      schema_file=f'{schema_path}/schema.pbtxt')
      
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_importer.outputs['schema'])

  components = [
      example_gen,
      statistics_gen,
      schema_importer,
      example_validator
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _test_schema_pipeline(
      pipeline_name=SCHEMA_PIPELINE_NAME,
      pipeline_root=SCHEMA_PIPELINE_ROOT,
      data_root=DATA_ROOT,
      schema_path=SCHEMA_FOLDER,
      metadata_path=SCHEMA_METADATA_PATH))

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Using deployment config:
 executor_specs {
  key: "CsvExampleGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.example_gen.csv_example_gen.executor.Executor"
      }
    }
  }
}
executor_specs {
  key: "ExampleValidator"
  value {
    python_class_executable_spec {
      class_path: "tfx.components.example_validator.executor.Executor"
    }
  }
}
executor_specs {
  key: "ImportSchemaGen"
  value {
    python_class_executable_spec {
      class_path: "tfx.components.schema_gen.import_schema_gen.executor.Executor"
    }
  }
}
executor_specs {
  key: "StatisticsGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.statistics_gen.executor.Executor"
      }
    }
  }
}
custom_driver_specs {
  key: "CsvExampleGen"
  value {
    python_class_execu

In [ ]:
metadata_connection_config_test = tfx.orchestration.metadata.sqlite_metadata_connection_config(SCHEMA_METADATA_PATH)

with Metadata(metadata_connection_config_test) as metadata_handler:
  # Find output artifacts from MLMD

  schema_importer_output = get_latest_artifacts(metadata_handler, SCHEMA_PIPELINE_NAME, 'ImportSchemaGen')
  schema_artifacts = schema_importer_output[standard_component_specs.SCHEMA_KEY]

INFO:absl:MetadataStore with DB connection initialized


In [ ]:
visualize_artifacts(schema_artifacts)

,Type,Presence,Valency,Domain
Feature name,,,,
'Residence_type',STRING,required,,'Residence_type'
'age',FLOAT,required,,min: 0.000000; max: 100.000000
'avg_glucose_level',FLOAT,required,,min: 25.000000; max: 300.000000
'bmi',BYTES,required,,min: 0.000000; max: 200.000000
'ever_married',STRING,required,,'ever_married'
'gender',STRING,required,,'gender'
'heart_disease',INT,required,,min: 0; max: 1
'hypertension',INT,required,,min: 0; max: 1
'id',INT,required,,-


,Values
Domain,
'Residence_type',"'Rural', 'Urban'"
'ever_married',"'No', 'Yes'"
'gender',"'Female', 'Male', 'Other'"
'smoking_status',"'Unknown', 'formerly smoked', 'never smoked', 'smokes'"
'work_type',"'Govt_job', 'Never_worked', 'Private', 'Self-employed', 'children'"


In [ ]:
# Copy original dataset
train_test_num = train_test.copy()
train_test_cat = train_test.copy()

# Categorical columns
cat_columns = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

# Label column
label_column = ['stroke']

# Drop the categorical and label columns
train_test_num.drop(cat_columns+label_column+['id'], axis=1, inplace=True)

# Only get categorical columns
train_test_cat = train_test_cat[cat_columns]

# Preview the resuls
train_test_num.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi
3448,55.0,0,0,92.59,36.6
4380,54.0,0,0,141.37,23.5
1515,82.0,0,0,58.30,20.4
2922,77.0,0,0,60.77,23.0
428,40.0,0,0,139.90,31.7


In [ ]:
# Preview the resuls
train_test_cat.head()

,gender,ever_married,work_type,Residence_type,smoking_status
3448,Male,Yes,Govt_job,Urban,never smoked
4380,Male,Yes,Private,Urban,never smoked
1515,Female,Yes,Govt_job,Rural,never smoked
2922,Male,Yes,Self-employed,Urban,smokes
428,Female,Yes,Private,Rural,smokes


In [ ]:
# Set the target values
y = train_test['stroke'].values

# Set the input values
X_num = train_test_num.values
X_cat = train_test_cat.values

In [ ]:
# Create SelectKBest object using f_classif (ANOVA statistics) and chi2 (Chi-Squared) for 4 features
select_k_best_num = SelectKBest(f_classif, k=4)
select_k_best_cat = SelectKBest(chi2, k=4)

# Fit and transform the input data using select_k_best
X_new_num = select_k_best_num.fit_transform(X_num, y)
ord = OrdinalEncoder()
X_ohe = ord.fit_transform(X_cat)
X_new_cat = select_k_best_cat.fit_transform(X_ohe, y)

# Extract the features which are selected using get_support API
num_features_mask = select_k_best_num.get_support()
cat_features_mask = select_k_best_cat.get_support()

# Print the results
num_reqd_cols = pd.DataFrame({'Columns': train_test_num.columns, 'Retain': num_features_mask})
cat_reqd_cols = pd.DataFrame({'Columns': train_test_cat.columns, 'Retain': cat_features_mask})
reqd_cols = num_reqd_cols.append(cat_reqd_cols, ignore_index=True)

print(reqd_cols)

             Columns  Retain
0                age    True
1       hypertension    True
2      heart_disease    True
3  avg_glucose_level    True
4                bmi   False
5             gender    True
6       ever_married    True
7          work_type    True
8     Residence_type   False
9     smoking_status    True


In [ ]:
# Get the feature names from SelectKBest
num_feature_names = list(train_test_num.columns[num_features_mask])
cat_feature_names = list(train_test_cat.columns[cat_features_mask])

# Append the label columns
feature_names = num_feature_names + cat_feature_names + label_column

# Select the selected subset of columns
train_test_select = train_test[feature_names]

# Write CSV to the created directory
train_test_select.to_csv(TRAINING_DATA_SELECT, index=False)

# Preview the results
train_test_select.head()

,age,hypertension,heart_disease,avg_glucose_level,gender,ever_married,work_type,smoking_status,stroke
3448,55.0,0,0,92.59,Male,Yes,Govt_job,never smoked,0
4380,54.0,0,0,141.37,Male,Yes,Private,never smoked,0
1515,82.0,0,0,58.30,Female,Yes,Govt_job,never smoked,0
2922,77.0,0,0,60.77,Male,Yes,Self-employed,smokes,0
428,40.0,0,0,139.90,Female,Yes,Private,smokes,0


In [ ]:
# Location of the pipeline metadata store
PIPELINE_DIR = f'{DATA_DIR}/pipeline'

# Declare the InteractiveContext and use a local sqlite file as the metadata store.
context = InteractiveContext(pipeline_root=PIPELINE_DIR)

In [ ]:
# Instantiate ExampleGen with the input CSV dataset
example_gen = tfx.components.CsvExampleGen(input_base=DATA_DIR_SELECT)

# Run the component using the InteractiveContext instance
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 47
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [ ]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
    
# Run the component
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 48
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [ ]:
# Display the results
context.show(statistics_gen.outputs['statistics'])

In [ ]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'])

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 49
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [ ]:
# Visualize the output
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',FLOAT,required,,-
'avg_glucose_level',FLOAT,required,,-
'ever_married',STRING,required,,'ever_married'
'gender',STRING,required,,'gender'
'heart_disease',INT,required,,-
'hypertension',INT,required,,-
'smoking_status',STRING,required,,'smoking_status'
'stroke',INT,required,,-
'work_type',STRING,required,,'work_type'


,Values
Domain,
'ever_married',"'No', 'Yes'"
'gender',"'Female', 'Male', 'Other'"
'smoking_status',"'Unknown', 'formerly smoked', 'never smoked', 'smokes'"
'work_type',"'Govt_job', 'Never_worked', 'Private', 'Self-employed', 'children'"


In [ ]:
# Getting the schema uri
schema_uri = schema_gen.outputs['schema']._artifacts[0].uri

In [ ]:
# Get the schema pbtxt file from the SchemaGen output
schema = tfdv.load_schema_text(os.path.join(schema_uri, 'schema.pbtxt'))

In [ ]:
# Set the `age` feature to an appropriate range
tfdv.set_domain(schema, 'age', schema_pb2.FloatDomain(name='age', min=0, max=100))

# Set the `avg_glucose_level` feature to an appropriate range
tfdv.set_domain(schema, 'avg_glucose_level', schema_pb2.FloatDomain(name='avg_glucose_level', min=25.0, max=300.0))

# Set the `heart_disease`, `stroke`, and `hypertension` features to 0/1 and make it categorical
tfdv.set_domain(schema, 'heart_disease', schema_pb2.IntDomain(name='heart_disease', min=0, max=1, is_categorical=True))
tfdv.set_domain(schema, 'hypertension', schema_pb2.IntDomain(name='hypertension', min=0, max=1, is_categorical=True))
tfdv.set_domain(schema, 'stroke', schema_pb2.IntDomain(name='stroke', min=0, max=1, is_categorical=True))

tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'age',FLOAT,required,,min: 0.000000; max: 100.000000
'avg_glucose_level',FLOAT,required,,min: 25.000000; max: 300.000000
'ever_married',STRING,required,,'ever_married'
'gender',STRING,required,,'gender'
'heart_disease',INT,required,,min: 0; max: 1
'hypertension',INT,required,,min: 0; max: 1
'smoking_status',STRING,required,,'smoking_status'
'stroke',INT,required,,min: 0; max: 1
'work_type',STRING,required,,'work_type'


,Values
Domain,
'ever_married',"'No', 'Yes'"
'gender',"'Female', 'Male', 'Other'"
'smoking_status',"'Unknown', 'formerly smoked', 'never smoked', 'smokes'"
'work_type',"'Govt_job', 'Never_worked', 'Private', 'Self-employed', 'children'"


In [ ]:
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

# Removing stroke feature from SERVING using TFDV
tfdv.get_feature(schema, 'stroke').not_in_environment.append('SERVING')

In [ ]:
# Declare StatsOptions to use the curated schema
stats_options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)

# Compute the statistics of the serving dataset
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA, stats_options=stats_options)

# Validate the serving dataset statistics in the `SERVING` environment
anomalies = tfdv.validate_statistics(serving_stats, schema=schema, environment='SERVING')

# Display the anomalies detected
tfdv.display_anomalies(anomalies)

In [ ]:
# Declare the path to the updated schema directory
UPDATED_SCHEMA_DIR = f'{PIPELINE_DIR}/updated_schema'

# Declare the path to the schema file
schema_file = os.path.join(UPDATED_SCHEMA_DIR, 'schema.pbtxt')

# Save the curated schema to the said file
tfdv.write_schema_text(schema, schema_file)

In [ ]:
# Load the schema from the directory we just created
new_schema = tfdv.load_schema_text(schema_file)

# Display the schema. Check that the Domain column still contains the ranges.
tfdv.display_schema(schema=new_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'age',FLOAT,required,,min: 0.000000; max: 100.000000
'avg_glucose_level',FLOAT,required,,min: 25.000000; max: 300.000000
'ever_married',STRING,required,,'ever_married'
'gender',STRING,required,,'gender'
'heart_disease',INT,required,,min: 0; max: 1
'hypertension',INT,required,,min: 0; max: 1
'smoking_status',STRING,required,,'smoking_status'
'stroke',INT,required,,min: 0; max: 1
'work_type',STRING,required,,'work_type'


,Values
Domain,
'ever_married',"'No', 'Yes'"
'gender',"'Female', 'Male', 'Other'"
'smoking_status',"'Unknown', 'formerly smoked', 'never smoked', 'smokes'"
'work_type',"'Govt_job', 'Never_worked', 'Private', 'Self-employed', 'children'"


In [ ]:
new_schema.default_environment

['TRAINING', 'SERVING']

In [ ]:
# Use ImportSchemaGen to put the curated schema to ML Metadata
user_schema_importer = tfx.components.ImportSchemaGen(schema_file=schema_file)

# Run the component
context.run(user_schema_importer, enable_cache=False)

context.show(user_schema_importer.outputs['schema'])

AlreadyExistsError: ignored

In [ ]:
# Use StatisticsGen to compute the statistics using the curated schema
statistics_gen_updated = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'],
    schema=user_schema_importer.outputs['schema'])

# Run the component
context.run(statistics_gen_updated)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 44
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [ ]:
context.show(statistics_gen_updated.outputs['statistics'])

In [ ]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen_updated.outputs['statistics'],
    schema=user_schema_importer.outputs['schema'])

# Run the component.
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 45
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [ ]:
# Visualize the results
context.show(example_validator.outputs['anomalies'])

In [ ]:
# Set the constants module filename
_stroke_constants_module_file = 'stroke_constants.py'

In [ ]:
%%writefile {_stroke_constants_module_file}

SCALE_MINMAX_FEATURE_KEYS = ['age']

SCALE_Z_FEATURE_KEYS = ['avg_glucose_level']

VOCAB_FEATURE_KEYS = ['ever_married', 'gender', 'heart_disease', 'hypertension', 'work_type']

LABEL_KEY = 'stroke'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Writing stroke_constants.py


In [ ]:
# Set the transform module filename
_stroke_transform_module_file = 'stroke_transform.py'

In [ ]:
%%writefile {_stroke_transform_module_file}

import stroke_constants

_SCALE_MINMAX_FEATURE_KEYS = stroke_constants.SCALE_MINMAX_FEATURE_KEYS
_SCALE_Z_FEATURE_KEYS = stroke_constants.SCALE_Z_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = stroke_constants.VOCAB_FEATURE_KEYS
_FEATURE_KEYS = _SCALE_MINMAX_FEATURE_KEYS + _SCALE_Z_FEATURE_KEYS + _VOCAB_FEATURE_KEYS
_LABEL_KEY = stroke_constants.LABEL_KEY
_transformed_name = stroke_constants.transformed_name

def preprocessing_fn(inputs):

    features_dict = {}

    for feature in _SCALE_MINMAX_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transform using scaling of min_max function
        features_dict[_transformed_name(feature)] = tft.scale_by_min_max(data_col)

    for feature in _SCALE_Z_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transforming using scaling to z score
        features_dict[_transformed_name(feature)] = tft.scale_to_z_score(data_col)

    for feature in _VOCAB_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transforming using vocabulary available in column
        features_dict[_transformed_name(feature)] = tft.compute_and_apply_vocabulary(data_col)

    # No change in the label
    features_dict[_LABEL_KEY] = inputs[_LABEL_KEY]

    return features_dict

def _apply_preprocessing(raw_features, tft_layer):

  transformed_features = tft_layer(raw_features)
  
  if _LABEL_KEY in raw_features:
    transformed_label = transformed_features.pop(_LABEL_KEY)
    return transformed_features, transformed_label
  
  else:
    return transformed_features, None

# This function will create a handler function which gets a serialized tf.example, preprocess and run an inference with it
def _get_serve_tf_examples_fn(model, tf_transform_output):
  # We must save the tft_layer to the model to ensure its assets are kept and tracked
  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_examples):
    # Expected input is a string which is serialized tf.Example format
    feature_spec = tf_transform_output.raw_feature_spec()
    # Because input schema can include unnecessary fields, we filter feature_spec to include required keys only
    required_feature_spec = {k: v for k, v in feature_spec.items() if k in _FEATURE_KEYS}
    parsed_features = tf.io.parse_example(serialized_tf_examples, required_feature_spec)

    # Preprocess parsed input with transform operation defined in preprocessing_fn()
    transformed_features, _ = _apply_preprocessing(parsed_features, model.tft_layer)
    
    # Run inference with ML model
    return model(transformed_features)

  return serve_tf_examples_fn

Writing stroke_transform.py


In [ ]:
# Instantiate the Transform component
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=user_schema_importer.outputs['schema'],
    module_file=os.path.abspath(_stroke_transform_module_file))

# Run the component
context.run(transform, enable_cache=False)

ExecutionResult(
    component_id: Transform
    execution_id: 46
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [ ]:
transform_uri = transform.outputs['transformed_examples'].get()[0].uri

In [ ]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform_uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [ ]:
# Get 3 records from the dataset and print the output to test

def get_records(dataset, num_records):
    """
    Extracts records from the given dataset.

    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    Returns:
        records: list of dictionaries of the row values
    """
    
    records = []
    
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))        
        # append to the records list
        records.append(example_dict)
      
    return records

transformed_dataset_subset = get_records(transformed_dataset, 3)
pp.pprint(transformed_dataset_subset)

[{'features': {'feature': {'age_xf': {'floatList': {'value': [0.59716797]}},
                           'avg_glucose_level_xf': {'floatList': {'value': [1.5289153]}},
                           'ever_married_xf': {'int64List': {'value': ['0']}},
                           'gender_xf': {'int64List': {'value': ['0']}},
                           'heart_disease_xf': {'int64List': {'value': ['0']}},
                           'hypertension_xf': {'int64List': {'value': ['0']}},
                           'stroke': {'int64List': {'value': ['1']}},
                           'work_type_xf': {'int64List': {'value': ['0']}}}}},
 {'features': {'feature': {'age_xf': {'floatList': {'value': [0.90234375]}},
                           'avg_glucose_level_xf': {'floatList': {'value': [-0.7955847]}},
                           'ever_married_xf': {'int64List': {'value': ['0']}},
                           'gender_xf': {'int64List': {'value': ['1']}},
                           'heart_disease_xf': {'int6